In [ ]:
# Script 3: Hyperparameter Optimization
# ==============================================================================
# --- User Configuration ---
# ==============================================================================
# Please modify all tunable parameters in this section.

# --- 1. Data Source and Splitting Settings ---
# Specify the path to your Excel file
EXCEL_FILE_PATH = 'Gap-final dataset.xlsx' # For target1
# EXCEL_FILE_PATH = 'Barrier-final dataset.xlsx' # For target2

# Define how to slice features (X) and the target (Y) from the data using column index positions.
# slice(start, stop) -> 'start' is the beginning index (inclusive), 'stop' is the ending index (exclusive).
X_COLS_SLICE = slice(1, -1)  # Example: X is from the 2nd column to the second-to-last column.
Y_COLS_SLICE = -1            # Example: Y is the last column.

# --- 2. Cross-Validation Settings ---
CV_N_SPLITS = 10
CV_SHUFFLE = True
CV_RANDOM_STATE = 100

# --- 3. Bayesian Optimization Settings ---
N_ITER_BAYESIAN = 50

# --- 4. General Model Settings ---
DEFAULT_MODEL_RANDOM_STATE = 0

# --- 5. Model Selection ---
# Select the models to run here.
# 【Uncomment】 a line to 【enable】 a model.
# 【Comment out】 a line to 【disable】 a model.
ENABLED_MODELS = [
    'XGBR',   # XGBoost
    'RF',     # Random Forest
    'GBRT',   # Gradient Boosting Regressor
    'HGBR',   # Hist Gradient Boosting
    'ETR',    # Extra Trees Regressor
    'CBR',    # CatBoost
    'LGBM',   # LightGBM
]


# ==============================================================================
# --- Model Hyperparameter Search Space Definitions ---
# ==============================================================================
# This section defines the hyperparameter optimization range for each model.
from skopt.space import Real, Integer, Categorical

parameter_XGBR = {
    'n_estimators': Integer(10, 3000), 'learning_rate': Real(0.01, 0.8, prior='log-uniform'),
    'max_depth': Integer(2, 30), 'subsample': Real(0.2, 1.0, prior='uniform'),
    'colsample_bytree': Real(0.2, 1.0, prior='uniform'), 'gamma': Real(0, 0.5, prior='uniform'),
    'reg_alpha': Real(1e-3, 1.0, prior='log-uniform'), 'reg_lambda': Real(0.1, 10.0, prior='log-uniform')
}
parameter_RF = {
    'n_estimators': Integer(10, 3000), 'max_depth': Integer(2, 30),
    'max_features': Categorical(['sqrt', 'log2']), 'min_samples_leaf': Integer(1, 20),
    'min_samples_split': Integer(2, 20)
}
parameter_CBR = {
    'iterations': Integer(10, 3000), 'learning_rate': Real(0.01, 0.8, prior='log-uniform'),
    'depth': Integer(1, 16), 'l2_leaf_reg': Real(1, 10, prior='log-uniform'),
    'subsample': Real(0.2, 1.0, prior='uniform'), 'rsm': Real(0.2, 1.0, prior='uniform')
}
parameter_LGBM = {
    'n_estimators': Integer(10, 3000), 'learning_rate': Real(0.01, 0.8, prior='log-uniform'),
    'max_depth': Integer(2, 50), 'num_leaves': Integer(5, 100),
    'subsample': Real(0.2, 1.0, prior='uniform'), 'colsample_bytree': Real(0.2, 1.0, prior='uniform'),
    'reg_alpha': Real(1e-3, 1.0, prior='log-uniform'), 'reg_lambda': Real(1e-3, 1.0, prior='log-uniform')
}
parameter_GBRT = {
    'n_estimators': Integer(10, 2000), 'learning_rate': Real(0.01, 0.6, prior='log-uniform'),
    'max_depth': Integer(2, 20), 'max_features': Categorical(['sqrt', 'log2']),
    'min_samples_split': Integer(2, 20), 'min_samples_leaf': Integer(1, 20),
    'subsample': Real(0.2, 1.0, prior='uniform')
}
parameter_HGBR = {
    'learning_rate': Real(0.01, 0.8, prior='log-uniform'), 'max_iter': Integer(10, 3000),
    'max_depth': Integer(2, 20), 'min_samples_leaf': Integer(1, 20),
    'l2_regularization': Real(1e-6, 10.0, prior='log-uniform')
}
parameter_ETR = {
    'n_estimators': Integer(10, 3000), 'max_depth': Integer(2, 30),
    'min_samples_split': Integer(2, 20), 'min_samples_leaf': Integer(1, 20),
    'max_features': Categorical(['sqrt', 'log2', 1.0])
}

# ==============================================================================
# --- Main Script Body (Usually no modification is needed below) ---
# ==============================================================================

# --- 1. Library Imports and Environment Check ---
import time
import pandas as pd
import numpy as np
import warnings
import re
import xgboost as XGB
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, ExtraTreesRegressor
from sklearn.model_selection import KFold
from skopt import BayesSearchCV

print("--- Script Start ---")

try:
    from catboost import CatBoostRegressor
    catboost_available = True
except ImportError:
    catboost_available = False
try:
    from lightgbm import LGBMRegressor
    lightgbm_available = True
except ImportError:
    lightgbm_available = False
print("Library imports and environment check complete.")


# --- 2. Data Loading and Preprocessing ---
X, Y = pd.DataFrame(), pd.Series()
try:
    print(f"\nLoading data from Excel file: {EXCEL_FILE_PATH}")
    df = pd.read_excel(EXCEL_FILE_PATH)
    print("File loaded successfully.")

    min_cols_required = 2
    if isinstance(X_COLS_SLICE, slice):
        min_cols_required = max(min_cols_required, abs(X_COLS_SLICE.start or 0), abs(X_COLS_SLICE.stop or 0))
    if df.shape[1] < min_cols_required:
         raise ValueError(f"Error: The number of columns in the data is insufficient for slicing with X_COLS_SLICE={X_COLS_SLICE}.")

    X = df.iloc[:, X_COLS_SLICE]
    Y = df.iloc[:, Y_COLS_SLICE]
    print(f"Data slicing complete. X shape={X.shape}, Y shape={Y.shape} (Target column: '{Y.name}')")

    print("\nCleaning feature names in X for XGBoost compatibility...")
    X.columns = [re.sub(r'\[|\]|<', '_', col) for col in X.columns]
    print("Column names cleaned.")

except FileNotFoundError:
    print(f"\n!!! Fatal Error: File not found at '{EXCEL_FILE_PATH}'. Please check the path in the configuration section.")
    exit()
except Exception as e:
    print(f"\n!!! Fatal Error: An error occurred while loading or processing data: {e}")
    exit()

# --- 3. Model and Cross-Validation Setup ---
cross_Valid = KFold(n_splits=CV_N_SPLITS, shuffle=CV_SHUFFLE, random_state=CV_RANDOM_STATE)

# Define all possible model instances
ALL_POSSIBLE_ESTIMATORS = {
    'XGBR': XGB.XGBRegressor(random_state=DEFAULT_MODEL_RANDOM_STATE, objective='reg:squarederror'),
    'RF': RandomForestRegressor(random_state=DEFAULT_MODEL_RANDOM_STATE),
    'GBRT': GradientBoostingRegressor(random_state=DEFAULT_MODEL_RANDOM_STATE),
    'HGBR': HistGradientBoostingRegressor(random_state=DEFAULT_MODEL_RANDOM_STATE),
    'ETR': ExtraTreesRegressor(random_state=DEFAULT_MODEL_RANDOM_STATE)
}
if catboost_available:
    ALL_POSSIBLE_ESTIMATORS['CBR'] = CatBoostRegressor(verbose=False, random_state=DEFAULT_MODEL_RANDOM_STATE, allow_writing_files=False)
else:
    print("Info: CatBoost is not installed. 'CBR' will be ignored if enabled in the configuration.")

if lightgbm_available:
    ALL_POSSIBLE_ESTIMATORS['LGBM'] = LGBMRegressor(random_state=DEFAULT_MODEL_RANDOM_STATE, verbosity=-1, objective='regression')
else:
    print("Info: LightGBM is not installed. 'LGBM' will be ignored if enabled in the configuration.")

# Build the list of models to run based on the user's selection in the configuration
estimators_for_bayes = {name: ALL_POSSIBLE_ESTIMATORS[name]
                        for name in ENABLED_MODELS
                        if name in ALL_POSSIBLE_ESTIMATORS}

params_mapping = {
    'XGBR': parameter_XGBR, 'RF': parameter_RF, 'GBRT': parameter_GBRT,
    'HGBR': parameter_HGBR, 'ETR': parameter_ETR
}
if catboost_available: params_mapping['CBR'] = parameter_CBR
if lightgbm_available: params_mapping['LGBM'] = parameter_LGBM

if not estimators_for_bayes:
    print("\n!!! Fatal Error: No models are enabled for tuning. Please check if the ENABLED_MODELS list in the configuration is empty or fully commented out.")
    exit()

print(f"\nSetup complete. The following enabled models will be tuned using {cross_Valid.get_n_splits()}-fold cross-validation: {list(estimators_for_bayes.keys())}")


# --- 4. Run Bayesian Optimization ---
grid_searches = {}
print(f"\nStarting BayesSearchCV for each model (n_iter = {N_ITER_BAYESIAN})...")

for name, estimator in estimators_for_bayes.items():
    start_time = time.time()
    print(f"\n--- Tuning {name} ---")
    if name not in params_mapping:
        print(f"Warning: Parameter space for model {name} not found. Skipping.")
        grid_searches[name] = None
        continue

    bayes_search = BayesSearchCV(
        estimator=estimator, search_spaces=params_mapping[name],
        n_iter=N_ITER_BAYESIAN, scoring='r2', cv=cross_Valid,
        n_jobs=-1, random_state=DEFAULT_MODEL_RANDOM_STATE, verbose=1
    )
    try:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UserWarning)
            bayes_search.fit(X, Y)
        duration = time.time() - start_time
        grid_searches[name] = bayes_search
        print(f"--- {name} tuning complete ---")
        print(f"  Best Score (CV R²): {bayes_search.best_score_:.4f}")
        print(f"  Best Parameters: {dict(bayes_search.best_params_)}")
        print(f"  Total tuning time: {duration:.2f} seconds")
    except Exception as e:
        duration = time.time() - start_time
        print(f"\n!!! An error occurred while tuning {name}: {e}")
        print(f"  Time spent on {name} tuning attempt: {duration:.2f} seconds")
        grid_searches[name] = None

# ==============================================================================
# --- 5. Final Summary of Best Hyperparameters ---
# ==============================================================================
print("\n\n==============================================================================")
print("--- Summary of All Model Tuning Results ---")
print("==============================================================================")

# Create a dictionary to store the best parameters for all models for potential export
all_best_params_for_export = {}

# Iterate through all completed search tasks
for name, search_result in grid_searches.items():
    print(f"\n--- Model: {name} ---")
    if search_result:
        # Extract the best score and parameters
        best_score = search_result.best_score_
        best_params = dict(search_result.best_params_)
        all_best_params_for_export[name] = best_params

        print(f"  Best R² Score (CV): {best_score:.4f}")
        print("  Best Hyperparameters:")
        # For better readability, print each parameter on a new line
        for param, value in best_params.items():
            # Format floats to 6 decimal places
            if isinstance(value, float):
                print(f"    - {param}: {value:.6f}")
            else:
                print(f"    - {param}: {value}")
    else:
        # If the model tuning failed or was skipped, display an info message
        print("  Tuning failed or was skipped.")
print("\n--- Script End ---")

In [ ]:
# Script 4: Stacking Ensemble & Evaluation
# ==============================================================================
# --- User Configuration ---
# ==============================================================================
# Please modify all tunable parameters in this section.

# --- 1. Evaluation Process Settings ---
# Number of seeds for repeated external cross-validation (how many times to run the full Stacking evaluation).
N_SEEDS_FOR_EVALUATION = 2
# Number of folds for the external cross-validation (used for generating OOF predictions and evaluating Stacking).
N_SPLITS_OUTER_CV = 10
# Strength factor for data augmentation (Gaussian noise). Set to 0 to disable.
NOISE_SCALE_FACTOR = 0

# --- 2. Meta-Learner Tuning Settings ---
# Number of iterations for Meta-Learner Bayesian Optimization (only performed on the first run, i.e., seed=0).
META_LEARNER_N_ITER_BAYESIAN = 50
# Number of 【splits】 for the Meta-Learner's internal cross-validation (e.g., 5).
META_LEARNER_N_SPLITS_CV = 10
# Number of 【repeats】 for the Meta-Learner's internal cross-validation (e.g., 1).
META_LEARNER_N_REPEATS_CV = 3

# --- 3. Plotting and Export Settings ---
# Number of top features to display in the feature importance bar chart.
N_FEATURES_TO_PLOT = 30
# Whether to plot the SHAP swarm plot (this can be time-consuming).
PLOT_SHAP_SWARM_PLOT = True
# Maximum number of samples for the SHAP swarm plot to prevent memory issues. Set to None for no limit.
SHAP_SWARM_SAMPLES_LIMIT = 5000

# --- 4. Base Learner Selection ---
# Select the models to be used as base learners in the Stacking ensemble.
# 【Uncomment】 a line to 【enable】 a model.
# 【Comment out】 a line to 【disable】 a model.
ENABLED_BASE_LEARNERS = [
    'XGBR',   # XGBoost
    'RF',     # Random Forest
    'GBRT',   # Gradient Boosting Regressor
    'HGBR',   # Hist Gradient Boosting
    'ETR',    # Extra Trees Regressor
    'CBR',    # CatBoost
    'LGBM',   # LightGBM
]

# --- 5. General Random State Settings ---
# Used for all models and cross-validation to ensure reproducibility.
DEFAULT_MODEL_RANDOM_STATE = 0


# ==============================================================================
# --- Main Script Body (Usually no modification is needed below) ---
# ==============================================================================

# --- 1. Library Imports and Environment Check ---
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import shap
import os
import traceback

import xgboost as XGB
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.base import clone
from skopt import BayesSearchCV
from skopt.space import Real

# Ignore some common warnings to make the output cleaner.
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

print("--- Script Start ---")

try:
    from catboost import CatBoostRegressor
    catboost_available = True
except ImportError:
    catboost_available = False
try:
    from lightgbm import LGBMRegressor
    lightgbm_available = True
except ImportError:
    lightgbm_available = False
print("Library imports and environment check complete.")


# --- 2. Core Variable Loading and Preparation ---
# Check if variables from the previous script exist; use mock data if not.
if 'X' not in locals() and 'X' not in globals():
    print("Warning: Variable 'X' is not defined. Using mock data.")
    X = pd.DataFrame(np.random.rand(100, 10), columns=[f'Feature_{i}' for i in range(10)])
if 'Y' not in locals() and 'Y' not in globals():
    print("Warning: Variable 'Y' is not defined. Using mock data.")
    Y = pd.Series(np.random.rand(100))
if 'grid_searches' not in locals() and 'grid_searches' not in globals():
    print("Warning: Variable 'grid_searches' is not defined. Using default (unoptimized) models.")
    grid_searches = {
        'XGBR': type('obj', (object,), {'best_estimator_': XGB.XGBRegressor(random_state=DEFAULT_MODEL_RANDOM_STATE, objective='reg:squarederror')})(),
        'RF': type('obj', (object,), {'best_estimator_': RandomForestRegressor(random_state=DEFAULT_MODEL_RANDOM_STATE)})(),
        'GBRT': type('obj', (object,), {'best_estimator_': GradientBoostingRegressor(random_state=DEFAULT_MODEL_RANDOM_STATE)})(),
        'ETR': type('obj', (object,), {'best_estimator_': ExtraTreesRegressor(random_state=DEFAULT_MODEL_RANDOM_STATE)})(),
        'HGBR': type('obj', (object,), {'best_estimator_': HistGradientBoostingRegressor(random_state=DEFAULT_MODEL_RANDOM_STATE)})()
    }
    if catboost_available:
        grid_searches['CBR'] = type('obj', (object,), {'best_estimator_': CatBoostRegressor(verbose=False, random_state=DEFAULT_MODEL_RANDOM_STATE, allow_writing_files=False)})()
    if lightgbm_available:
        grid_searches['LGBM'] = type('obj', (object,), {'best_estimator_': LGBMRegressor(random_state=DEFAULT_MODEL_RANDOM_STATE, verbosity=-1, objective='regression')})()

# Filter models to be used based on user configuration
filtered_grid_searches = {name: gs for name, gs in grid_searches.items() if name in ENABLED_BASE_LEARNERS and gs is not None}
print(f"\nModel filtering complete: Selected {len(filtered_grid_searches)} enabled models from {len(grid_searches)} available models: {list(filtered_grid_searches.keys())}")
if not filtered_grid_searches:
    print("\n!!! Fatal Error: No base learners were selected. Please check the ENABLED_BASE_LEARNERS list in the configuration.")
    exit()

# Data Format Check and Preparation
try:
    if not isinstance(X, pd.DataFrame):
        print("Warning: For accurate sample indexing, it is recommended that the input X is a Pandas DataFrame.")
        X_index = pd.RangeIndex(start=0, stop=len(X), step=1)
    else:
        X_index = X.index
    if len(Y) != len(X): raise ValueError("Length of X and Y do not match!")
    if isinstance(Y, pd.DataFrame):
        if Y.shape[1] != 1: raise ValueError("If Y is a DataFrame, it should contain only one target column.")
        if not Y.index.equals(X_index): print("Warning: The index of Y does not match the index of X.")
        Y = Y.iloc[:, 0]
except NameError as e:
    print(f"Error: {e}. Please ensure the optimization script has been run or the required variables are loaded.")
    exit()
except ValueError as e:
    print(f"Data Error: {e}")
    exit()

if NOISE_SCALE_FACTOR > 0:
    print(f"Gaussian noise augmentation is enabled with a strength factor of: {NOISE_SCALE_FACTOR}")
else:
    print("Noise augmentation is disabled.")
np.random.seed(DEFAULT_MODEL_RANDOM_STATE)


# --- 3. Core Function Definitions ---
def initialize_best_estimators(grid_searches_dict):
    """Initialize model instances from BayesSearchCV results."""
    estimators_init = {}
    available_models = {
        'XGBR': (XGB.XGBRegressor, {'objective': 'reg:squarederror', 'random_state': DEFAULT_MODEL_RANDOM_STATE}),
        'RF': (RandomForestRegressor, {'random_state': DEFAULT_MODEL_RANDOM_STATE}),
        'GBRT': (GradientBoostingRegressor, {'random_state': DEFAULT_MODEL_RANDOM_STATE}),
        'ETR': (ExtraTreesRegressor, {'random_state': DEFAULT_MODEL_RANDOM_STATE}),
        'HGBR': (HistGradientBoostingRegressor, {'random_state': DEFAULT_MODEL_RANDOM_STATE})
    }
    if catboost_available: available_models['CBR'] = (CatBoostRegressor, {'verbose': False, 'random_state': DEFAULT_MODEL_RANDOM_STATE, 'allow_writing_files': False})
    if lightgbm_available: available_models['LGBM'] = (LGBMRegressor, {'random_state': DEFAULT_MODEL_RANDOM_STATE, 'verbosity': -1, 'objective': 'regression'})

    print("Initializing models...")
    for name in grid_searches_dict.keys():
        if name not in available_models: continue
        model_class, fixed_params = available_models[name]
        if name in grid_searches_dict and hasattr(grid_searches_dict[name], 'best_estimator_'):
            try:
                estimators_init[name] = clone(grid_searches_dict[name].best_estimator_)
                print(f"  Successfully initialized {name} with optimized parameters.")
            except Exception as e:
                print(f"  Error initializing {name} from best_estimator_: {e}. This model will be skipped.")
                estimators_init[name] = None
        else:
            print(f"  Optimized results for {name} not found. Initializing with default parameters.")
            try:
                estimators_init[name] = model_class(**fixed_params)
                print(f"  Successfully initialized {name} with default parameters.")
            except Exception as e:
                print(f"  Error initializing {name} with default parameters: {e}. This model will be skipped.")
                estimators_init[name] = None
    
    initialized_estimators = {k: v for k, v in estimators_init.items() if v is not None}
    
    if not initialized_estimators: print("Warning: No models were successfully initialized!")
    return initialized_estimators

meta_learner_params = {
    'elasticnet__alpha': Real(1e-5, 10.0, prior='log-uniform', name='elasticnet__alpha'),
    'elasticnet__l1_ratio': Real(0.0, 1.0, prior='uniform', name='elasticnet__l1_ratio')
}
global_best_meta_learner = None
global_meta_learner_coeffs = None

def evaluate_models_manual_cv(seed, X_full, Y_full, X_full_index, grid_searches_dict, noise_factor, meta_learner_instance=None):
    """Manually execute one full cross-validation evaluation process."""
    print(f"\n--- Starting Seed {seed} ---")
    cross_validator = KFold(n_splits=N_SPLITS_OUTER_CV, shuffle=True, random_state=seed)
    best_estimators_dict = initialize_best_estimators(grid_searches_dict)
    if not best_estimators_dict:
        return {'submodel_r2_means': {}, 'submodel_rmse_means': {}, 'stacking_regressor_r2_mean': np.nan, 'stacking_regressor_rmse_mean': np.nan, 'plot_data': [], 'meta_learner_coefficients': {}, 'raw_shap_values': {}, 'shap_expected_values': {}, 'X_val_data_for_shap': [], 'all_scores_data_for_seed_fold': []}

    submodels = list(best_estimators_dict.items())
    submodel_names = [name for name, _ in submodels]
    fold_r2_scores = {name: [] for name in submodel_names}; fold_rmse_scores = {name: [] for name in submodel_names}
    fold_r2_scores['Stacking'] = []; fold_rmse_scores['Stacking'] = []
    all_plot_data_for_seed, all_scores_data_for_seed_fold = [], []
    
    meta_coefficients_sum_current_seed, meta_coefficients_count_current_seed = np.zeros(len(submodel_names)), 0
    all_raw_shap_values_per_model = {name: [] for name in submodel_names}
    all_shap_expected_values_per_model = {name: [] for name in submodel_names}
    all_X_val_data_for_shap_folds = []

    print(f"  Seed {seed}: Starting {N_SPLITS_OUTER_CV}-fold cross-validation (Outer CV)...")
    is_X_dataframe = isinstance(X_full, pd.DataFrame)
    original_columns = X_full.columns if is_X_dataframe else [f'Feature_{i}' for i in range(X_full.shape[1])]
    
    for fold_idx, (train_loc_idx, val_loc_idx) in enumerate(cross_validator.split(X_full, Y_full)):
        train_original_indices, val_original_indices = X_full_index[train_loc_idx], X_full_index[val_loc_idx]
        if is_X_dataframe:
            X_train_fold, X_val_fold = X_full.loc[train_original_indices], X_full.loc[val_original_indices]
            Y_train_fold, Y_val_fold = Y_full.loc[train_original_indices], Y_full.loc[val_original_indices]
        else:
            X_train_fold, X_val_fold = X_full[train_loc_idx], X_full[val_loc_idx]
            Y_train_fold, Y_val_fold = Y_full[train_loc_idx], Y_full[val_loc_idx]
        
        y_val_true_np = Y_val_fold.values if isinstance(Y_val_fold, pd.Series) else np.array(Y_val_fold)
        y_train_true_np = Y_train_fold.values if isinstance(Y_train_fold, pd.Series) else np.array(Y_train_fold)
        
        X_train_augmented = X_train_fold
        if noise_factor > 0:
            X_train_fold_np = X_train_fold.values if is_X_dataframe else X_train_fold
            noise = np.random.normal(loc=0.0, scale=np.std(X_train_fold_np, axis=0) * noise_factor + 1e-9, size=X_train_fold_np.shape)
            X_train_augmented = pd.DataFrame(X_train_fold_np + noise, index=train_original_indices, columns=original_columns) if is_X_dataframe else X_train_fold_np + noise
        
        oof_preds_train, oof_preds_val = np.zeros((len(Y_train_fold), len(submodels))), np.zeros((len(Y_val_fold), len(submodels)))
        all_X_val_data_for_shap_folds.append(X_val_fold)

        for i, (name, estimator_template) in enumerate(submodels):
            estimator_fold = clone(estimator_template)
            try:
                estimator_fold.fit(X_train_augmented, Y_train_fold)
            except Exception as fit_e:
                print(f"\nWarning: Seed {seed}, Fold {fold_idx+1}, model {name} failed to fit: {fit_e}")
                oof_preds_train[:, i], oof_preds_val[:, i] = 0, 0
                all_scores_data_for_seed_fold.extend([{'Seed': seed, 'Fold': fold_idx + 1, 'Model': name, 'Metric': 'R2', 'Value': np.nan}, {'Seed': seed, 'Fold': fold_idx + 1, 'Model': name, 'Metric': 'RMSE', 'Value': np.nan}])
                continue

            oof_preds_train[:, i] = estimator_fold.predict(X_train_fold)
            oof_preds_val[:, i] = estimator_fold.predict(X_val_fold)
            r2 = r2_score(Y_val_fold, oof_preds_val[:, i]); rmse = np.sqrt(mean_squared_error(Y_val_fold, oof_preds_val[:, i]))
            fold_r2_scores[name].append(r2); fold_rmse_scores[name].append(rmse)
            all_scores_data_for_seed_fold.extend([{'Seed': seed, 'Fold': fold_idx + 1, 'Model': name, 'Metric': 'R2', 'Value': r2}, {'Seed': seed, 'Fold': fold_idx + 1, 'Model': name, 'Metric': 'RMSE', 'Value': rmse}])
            
            for idx, (actual, pred) in enumerate(zip(y_val_true_np, oof_preds_val[:, i])): all_plot_data_for_seed.append({'Seed': seed, 'Fold': fold_idx + 1, 'Model': name, 'Set': 'Validation', 'Actual': actual, 'Predicted': pred, 'Sample_Index': val_original_indices[idx]})
            for idx, (actual, pred) in enumerate(zip(y_train_true_np, oof_preds_train[:, i])): all_plot_data_for_seed.append({'Seed': seed, 'Fold': fold_idx + 1, 'Model': name, 'Set': 'Train', 'Actual': actual, 'Predicted': pred, 'Sample_Index': train_original_indices[idx]})

            try:
                X_val_fold_np = X_val_fold.values if isinstance(X_val_fold, pd.DataFrame) else X_val_fold
                if X_val_fold_np.shape[0] == 0: continue
                if isinstance(estimator_fold, (XGB.XGBRegressor, RandomForestRegressor, GradientBoostingRegressor, CatBoostRegressor, LGBMRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor)):
                    explainer = shap.TreeExplainer(estimator_fold)
                    shap_values = explainer.shap_values(X_val_fold_np); expected_value = explainer.expected_value
                else: continue
                all_raw_shap_values_per_model[name].append(shap_values); all_shap_expected_values_per_model[name].append(expected_value)
            except Exception as imp_e: print(f"\nWarning: Seed {seed}, Fold {fold_idx+1}, error getting SHAP values for {name}: {imp_e}")

        if np.all(oof_preds_train == 0) or np.any(np.isnan(oof_preds_train)):
            print(f"Warning: Seed {seed}, Fold {fold_idx+1}, OOF predictions are invalid. Skipping meta-learner.")
            fold_r2_scores['Stacking'].append(np.nan); fold_rmse_scores['Stacking'].append(np.nan)
            all_scores_data_for_seed_fold.extend([{'Seed': seed, 'Fold': fold_idx + 1, 'Model': 'Stacking', 'Metric': 'R2', 'Value': np.nan}, {'Seed': seed, 'Fold': fold_idx + 1, 'Model': 'Stacking', 'Metric': 'RMSE', 'Value': np.nan}])
            continue

        current_meta_learner = None
        if meta_learner_instance:
            current_meta_learner = clone(meta_learner_instance)
            current_meta_learner.fit(oof_preds_train, Y_train_fold) 
        else:
            print(f"  Seed {seed}, Fold {fold_idx+1}: Tuning meta-learner...")
            res_meta = BayesSearchCV(estimator=Pipeline([('scaler', StandardScaler()), ('elasticnet', ElasticNet(random_state=DEFAULT_MODEL_RANDOM_STATE, max_iter=2000))]), search_spaces=meta_learner_params, n_iter=META_LEARNER_N_ITER_BAYESIAN, scoring='r2', cv=RepeatedKFold(n_splits=META_LEARNER_N_SPLITS_CV, n_repeats=META_LEARNER_N_REPEATS_CV, random_state=DEFAULT_MODEL_RANDOM_STATE), n_jobs=-1, random_state=DEFAULT_MODEL_RANDOM_STATE, verbose=0)
            try:
                res_meta.fit(oof_preds_train, Y_train_fold)
                current_meta_learner = res_meta.best_estimator_
            except Exception as meta_e: print(f"\nWarning: Seed {seed}, Fold {fold_idx+1}, meta-learner tuning failed: {meta_e}")

        if current_meta_learner:
            try:
                elastic_net_model = current_meta_learner.named_steps['elasticnet']
                meta_coefficients_sum_current_seed += elastic_net_model.coef_; meta_coefficients_count_current_seed += 1
                y_pred_val_stacking = current_meta_learner.predict(oof_preds_val); y_pred_train_stacking = current_meta_learner.predict(oof_preds_train)
                r2_stacking = r2_score(Y_val_fold, y_pred_val_stacking); rmse_stacking = np.sqrt(mean_squared_error(Y_val_fold, y_pred_val_stacking))
                fold_r2_scores['Stacking'].append(r2_stacking); fold_rmse_scores['Stacking'].append(rmse_stacking)
                all_scores_data_for_seed_fold.extend([{'Seed': seed, 'Fold': fold_idx + 1, 'Model': 'Stacking', 'Metric': 'R2', 'Value': r2_stacking}, {'Seed': seed, 'Fold': fold_idx + 1, 'Model': 'Stacking', 'Metric': 'RMSE', 'Value': rmse_stacking}])
                for idx, (actual, pred) in enumerate(zip(y_val_true_np, y_pred_val_stacking)): all_plot_data_for_seed.append({'Seed': seed, 'Fold': fold_idx + 1, 'Model': 'Stacking', 'Set': 'Validation', 'Actual': actual, 'Predicted': pred, 'Sample_Index': val_original_indices[idx]})
                for idx, (actual, pred) in enumerate(zip(y_train_true_np, y_pred_train_stacking)): all_plot_data_for_seed.append({'Seed': seed, 'Fold': fold_idx + 1, 'Model': 'Stacking', 'Set': 'Train', 'Actual': actual, 'Predicted': pred, 'Sample_Index': train_original_indices[idx]})
            except Exception as meta_e: 
                print(f"\nWarning: Seed {seed}, Fold {fold_idx+1}, meta-learner prediction failed: {meta_e}")
                fold_r2_scores['Stacking'].append(np.nan); fold_rmse_scores['Stacking'].append(np.nan)
        
    submodel_r2_means = {name: np.nanmean(scores) if scores else np.nan for name, scores in fold_r2_scores.items()}
    submodel_rmse_means = {name: np.nanmean(scores) if scores else np.nan for name, scores in fold_rmse_scores.items()}
    stacking_regressor_r2_mean = submodel_r2_means.pop('Stacking', np.nan)
    stacking_regressor_rmse_mean = submodel_rmse_means.pop('Stacking', np.nan)
    avg_meta_coefficients_current_seed = dict(zip(submodel_names, meta_coefficients_sum_current_seed / meta_coefficients_count_current_seed)) if meta_coefficients_count_current_seed > 0 else {}
    
    return {'submodel_r2_means': submodel_r2_means, 'submodel_rmse_means': submodel_rmse_means, 'stacking_regressor_r2_mean': stacking_regressor_r2_mean, 'stacking_regressor_rmse_mean': stacking_regressor_rmse_mean, 'plot_data': all_plot_data_for_seed, 'meta_learner_coefficients': avg_meta_coefficients_current_seed, 'raw_shap_values': all_raw_shap_values_per_model, 'shap_expected_values': all_shap_expected_values_per_model, 'X_val_data_for_shap': all_X_val_data_for_shap_folds, 'all_scores_data_for_seed_fold': all_scores_data_for_seed_fold}


# --- 4. Main Execution Flow ---
if isinstance(X, pd.DataFrame):
    feature_names_list = X.columns.tolist()
else:
    feature_names_list = [f'Feature_{i}' for i in range(X.shape[1])]

all_results, all_plot_data_accumulated, all_scores_accumulated = [], [], []
seeds = range(N_SEEDS_FOR_EVALUATION)
model_names_available = list(filtered_grid_searches.keys())
submodel_r2_sums = {name: 0.0 for name in model_names_available}; submodel_rmse_sums = {name: 0.0 for name in model_names_available}
stacking_r2_sum, stacking_rmse_sum = 0.0, 0.0
meta_coefficients_sums_across_seeds = {name: 0.0 for name in model_names_available}
valid_seeds_count = 0
all_aggregated_shap_values_for_swarm_raw_data = []

print(f"\n=== Starting Repeated Cross-Validation on {len(seeds)} Different Seeds ===")
for seed in seeds:
    if seed == 0:
        print("\n--- First Iteration (Seed 0): Tuning Meta-Learner Hyperparameters ---")
        oof_preds_full = np.zeros((len(X), len(model_names_available)))
        kf_full_data = KFold(n_splits=N_SPLITS_OUTER_CV, shuffle=True, random_state=seed)
        temp_base_estimators = initialize_best_estimators(filtered_grid_searches)
        if not temp_base_estimators: print("Warning: Could not initialize base learners, skipping meta-learner tuning."); break
        for i, (name, estimator_template) in enumerate(temp_base_estimators.items()):
            print(f"  Generating OOF predictions for model {name}...")
            fold_preds = np.zeros(len(X))
            for train_idx, val_idx in kf_full_data.split(X, Y):
                X_train_fold, X_val_fold = (X.iloc[train_idx], X.iloc[val_idx]) if isinstance(X, pd.DataFrame) else (X[train_idx], X[val_idx])
                Y_train_fold = Y.iloc[train_idx] if isinstance(Y, pd.Series) else Y[train_idx]
                estimator_clone = clone(estimator_template); estimator_clone.fit(X_train_fold, Y_train_fold)
                fold_preds[val_idx] = estimator_clone.predict(X_val_fold)
            oof_preds_full[:, i] = fold_preds
        
        if np.all(oof_preds_full == 0) or np.any(np.isnan(oof_preds_full)):
            print("Warning: Generated OOF predictions are invalid. Cannot tune meta-learner.")
            global_best_meta_learner, global_meta_learner_coeffs = None, {name: np.nan for name in model_names_available}
            result = evaluate_models_manual_cv(seed, X, Y, X_index, filtered_grid_searches, NOISE_SCALE_FACTOR, meta_learner_instance=None)
        else:
            print("  Starting Bayesian Optimization for the meta-learner...")
            meta_bayes_search = BayesSearchCV(
                estimator=Pipeline([('scaler', StandardScaler()), ('elasticnet', ElasticNet(random_state=DEFAULT_MODEL_RANDOM_STATE, max_iter=2000))]),
                search_spaces=meta_learner_params, n_iter=META_LEARNER_N_ITER_BAYESIAN, scoring='r2',
                cv=RepeatedKFold(n_splits=META_LEARNER_N_SPLITS_CV, n_repeats=META_LEARNER_N_REPEATS_CV, random_state=DEFAULT_MODEL_RANDOM_STATE),
                n_jobs=-1, random_state=DEFAULT_MODEL_RANDOM_STATE, verbose=1)
            try:
                meta_bayes_search.fit(oof_preds_full, Y)
                global_best_meta_learner = meta_bayes_search.best_estimator_
                elastic_net_model = global_best_meta_learner.named_steps['elasticnet']
                global_meta_learner_coeffs = dict(zip(model_names_available, elastic_net_model.coef_))
                print("\n--- Meta-Learner tuning complete ---"); print(f"Meta-Learner Best Score (CV R² on Full OOF): {meta_bayes_search.best_score_:.4f}"); print(f"Meta-Learner Best Parameters: {dict(meta_bayes_search.best_params_)}"); print(f"Meta-Learner Coefficients: {global_meta_learner_coeffs}")
                result = evaluate_models_manual_cv(seed, X, Y, X_index, filtered_grid_searches, NOISE_SCALE_FACTOR, meta_learner_instance=global_best_meta_learner)
            except Exception as e:
                print(f"\n!!! Meta-learner tuning failed: {e}")
                global_best_meta_learner, global_meta_learner_coeffs = None, {name: np.nan for name in model_names_available}
                result = evaluate_models_manual_cv(seed, X, Y, X_index, filtered_grid_searches, NOISE_SCALE_FACTOR, meta_learner_instance=None)
    else:
        print(f"\n--- Subsequent Iteration (Seed {seed}): Using pre-tuned meta-learner ---")
        result = evaluate_models_manual_cv(seed, X, Y, X_index, filtered_grid_searches, NOISE_SCALE_FACTOR, meta_learner_instance=global_best_meta_learner)

    all_results.append(result)
    if result.get('plot_data'): all_plot_data_accumulated.extend(result['plot_data'])
    if result.get('all_scores_data_for_seed_fold'): all_scores_accumulated.extend(result['all_scores_data_for_seed_fold'])
    
    print(f"\n--- Seed {seed} Results Summary ---")
    current_stacking_r2 = result.get('stacking_regressor_r2_mean', np.nan)
    current_stacking_rmse = result.get('stacking_regressor_rmse_mean', np.nan)
    sr_r2_str = f"{current_stacking_r2:.4f}" if not np.isnan(current_stacking_r2) else "N/A"
    sr_rmse_str = f"{current_stacking_rmse:.4f}" if not np.isnan(current_stacking_rmse) else "N/A"
    print(f"  Stacking Regressor R2 Mean: {sr_r2_str}"); print(f"  Stacking Regressor RMSE Mean: {sr_rmse_str}")
    print("  Submodel R2 and RMSE Means:")
    current_sub_r2 = result.get('submodel_r2_means', {})
    current_sub_rmse = result.get('submodel_rmse_means', {})
    for name in model_names_available:
        r2_val, rmse_val = current_sub_r2.get(name, np.nan), current_sub_rmse.get(name, np.nan)
        r2_str = f"{r2_val:.4f}" if not np.isnan(r2_val) else "N/A"; rmse_str = f"{rmse_val:.4f}" if not np.isnan(rmse_val) else "N/A"
        print(f"      {name}: R2 Mean = {r2_str}, RMSE Mean = {rmse_str}")
    if global_meta_learner_coeffs:
        print("  Meta-Learner Coefficients (Fixed after Seed 0):")
        for name, coeff in global_meta_learner_coeffs.items(): print(f"      {name}: {coeff:.4f}")
    
    if PLOT_SHAP_SWARM_PLOT and result.get('raw_shap_values') and result.get('X_val_data_for_shap'):
        for model_name in model_names_available:
            if model_name in result['raw_shap_values']:
                for i in range(len(result['raw_shap_values'][model_name])):
                    if i < len(result['X_val_data_for_shap']):
                        all_aggregated_shap_values_for_swarm_raw_data.append({'seed': seed, 'fold_idx': i, 'model_name': model_name, 'shap_values': result['raw_shap_values'][model_name][i], 'expected_value': result['shap_expected_values'][model_name][i], 'X_val_data': result['X_val_data_for_shap'][i]})

    if not np.isnan(current_stacking_r2):
        valid_seeds_count += 1
        stacking_r2_sum += current_stacking_r2
        stacking_rmse_sum += result.get('stacking_regressor_rmse_mean', np.nan)
        for name in model_names_available:
            submodel_r2_sums[name] += result.get('submodel_r2_means', {}).get(name, np.nan)
            submodel_rmse_sums[name] += result.get('submodel_rmse_means', {}).get(name, np.nan)
        if global_meta_learner_coeffs:
            for name, coeff in global_meta_learner_coeffs.items(): meta_coefficients_sums_across_seeds[name] += coeff
    else: print(f"Warning: Stacking Regressor result for Seed {seed} is NaN. Skipping accumulation.")

# --- 5. Final Results Visualization and Export ---
if valid_seeds_count > 0:
    avg_stacking_regressor_r2_mean = stacking_r2_sum / valid_seeds_count
    avg_stacking_regressor_rmse_mean = stacking_rmse_sum / valid_seeds_count
    avg_submodel_r2_means = {name: total / valid_seeds_count for name, total in submodel_r2_sums.items()}
    avg_submodel_rmse_means = {name: total / valid_seeds_count for name, total in submodel_rmse_sums.items()}
    avg_meta_coefficients_final = {name: total / valid_seeds_count for name, total in meta_coefficients_sums_across_seeds.items()} if global_meta_learner_coeffs else {name: np.nan for name in model_names_available}
    
    print(f"\n\n==============================================================================")
    print(f"--- Average Results Across All {valid_seeds_count} Valid Seeds ---")
    print(f"==============================================================================")
    print(f"\nAverage Stacking Regressor R2 Mean: {avg_stacking_regressor_r2_mean:.4f}")
    print(f"Average Stacking Regressor RMSE Mean: {avg_stacking_regressor_rmse_mean:.4f}")
    print("\nAverage Submodel R2 and RMSE Means:")
    for name in model_names_available:
        r2_avg, rmse_avg = avg_submodel_r2_means.get(name, np.nan), avg_submodel_rmse_means.get(name, np.nan)
        print(f"  {name}: R2 Mean = {r2_avg:.4f}, RMSE Mean = {rmse_avg:.4f}")
    
    print("\nAverage Meta-Learner Coefficients (Averaged over Seeds):")
    for name, coeff in avg_meta_coefficients_final.items(): print(f"  {name}: {coeff:.4f}")

    # =================================================================================
    # --- Two-Level Weighted SHAP Analysis Module ---
    # =================================================================================
    if PLOT_SHAP_SWARM_PLOT and all_aggregated_shap_values_for_swarm_raw_data and all_scores_accumulated:
        print("\n--- Performing [Two-Level Weighted] SHAP Aggregation and Feature Importance Analysis ---")
        print("    Level 1: Within each training fold, weight by the base model's R².")
        print("    Level 2: Across all training folds, weight by the Stacking model's R².")

        try:
            # 1. Prepare data
            scores_df = pd.DataFrame(all_scores_accumulated)
            scores_pivot = scores_df.pivot_table(index=['Seed', 'Fold'], columns=['Model', 'Metric'], values='Value')
            
            grouped_shap_data = {}
            for item in all_aggregated_shap_values_for_swarm_raw_data:
                key = (item['seed'], item['fold_idx'])
                if key not in grouped_shap_data:
                    grouped_shap_data[key] = {'models_data': {}, 'X_val_data': item['X_val_data']}
                grouped_shap_data[key]['models_data'][item['model_name']] = {'shap_values': item['shap_values'], 'expected_value': item['expected_value']}

            # 2. Perform two-level weighting
            level1_weighted_shap_list, level1_weighted_expected_values_list = [], []
            level1_X_data_list, level2_weights_list = [], []
            
            print("  Iterating through all Seeds and Folds to calculate weighted SHAP values...")
            for (seed, fold_idx), data in grouped_shap_data.items():
                models_data_in_fold = data['models_data']
                
                # Level 1 Weighting (within a fold)
                base_model_r2s = {}
                valid_models_in_fold = list(models_data_in_fold.keys())
                for model_name in valid_models_in_fold:
                    try:
                        r2_tuple = (model_name, 'R2')
                        r2 = scores_pivot.loc[(seed, fold_idx + 1), r2_tuple]
                        base_model_r2s[model_name] = max(0, r2) if not np.isnan(r2) else 0.0
                    except KeyError:
                        base_model_r2s[model_name] = 0.0

                total_r2_sum_level1 = sum(base_model_r2s.values())
                if total_r2_sum_level1 < 1e-9: continue

                first_valid_shap, first_valid_X = None, None
                for model_name in valid_models_in_fold:
                    if models_data_in_fold[model_name]['shap_values'].shape[0] > 0:
                        first_valid_shap = models_data_in_fold[model_name]['shap_values']
                        first_valid_X = data['X_val_data']
                        break
                if first_valid_shap is None: continue

                weighted_shap_for_fold = np.zeros_like(first_valid_shap)
                weighted_expected_value_for_fold = 0.0
                for model_name, model_data in models_data_in_fold.items():
                    if model_data['shap_values'].shape == weighted_shap_for_fold.shape:
                        weight = base_model_r2s.get(model_name, 0.0) / total_r2_sum_level1
                        weighted_shap_for_fold += weight * model_data['shap_values']
                        weighted_expected_value_for_fold += weight * model_data['expected_value']
                
                # Level 2 Weight (across folds)
                try:
                    stacking_r2_tuple = ('Stacking', 'R2')
                    level2_weight = scores_pivot.loc[(seed, fold_idx + 1), stacking_r2_tuple]
                    level2_weight = max(0, level2_weight) if not np.isnan(level2_weight) else 0.0
                except KeyError:
                    level2_weight = 0.0

                level1_weighted_shap_list.append(weighted_shap_for_fold)
                level1_weighted_expected_values_list.append(weighted_expected_value_for_fold)
                level1_X_data_list.append(first_valid_X)
                level2_weights_list.append(level2_weight)

            # 3. Aggregate all data and apply level 2 weighting
            if level1_weighted_shap_list:
                print("  Aggregating data from all folds and applying Level 2 weights...")
                final_shap_values_for_plot = np.vstack(level1_weighted_shap_list)
                final_X_data_for_plot = pd.concat(level1_X_data_list, ignore_index=True)
                
                sample_weights_level2 = np.repeat(level2_weights_list, [matrix.shape[0] for matrix in level1_weighted_shap_list])

                sample_indices = slice(None)
                if SHAP_SWARM_SAMPLES_LIMIT is not None and final_shap_values_for_plot.shape[0] > SHAP_SWARM_SAMPLES_LIMIT:
                    print(f"  Warning: Number of SHAP samples is too large ({final_shap_values_for_plot.shape[0]}). Subsampling to {SHAP_SWARM_SAMPLES_LIMIT}.")
                    rng = np.random.default_rng(DEFAULT_MODEL_RANDOM_STATE)
                    sample_indices = rng.choice(final_shap_values_for_plot.shape[0], SHAP_SWARM_SAMPLES_LIMIT, replace=False)
                    final_shap_values_for_plot = final_shap_values_for_plot[sample_indices]
                    final_X_data_for_plot = final_X_data_for_plot.iloc[sample_indices]
                    sample_weights_level2 = sample_weights_level2[sample_indices]

                if np.sum(sample_weights_level2) > 1e-9:
                    expected_values_per_sample = np.repeat(level1_weighted_expected_values_list, [matrix.shape[0] for matrix in level1_weighted_shap_list])
                    if isinstance(sample_indices, np.ndarray):
                        expected_values_per_sample = expected_values_per_sample[sample_indices]

                    final_base_value = np.average(expected_values_per_sample, weights=sample_weights_level2)

                    # 4. Calculate unified feature importance
                    print("\n--- Unified Feature Importance Results (from Two-Level Weighted SHAP data) ---")
                    unified_feature_importances = pd.Series(
                        np.average(np.abs(final_shap_values_for_plot), axis=0, weights=sample_weights_level2),
                        index=feature_names_list
                    )
                    sorted_feature_importances = sorted(unified_feature_importances.to_dict().items(), key=lambda item: item[1], reverse=True)
                    
                    print("Global Feature Importances (sorted):")
                    for feature, importance in sorted_feature_importances:
                        print(f"  {feature}: {importance:.4f}")

                    # 5. Plot unified feature importance bar chart
                    print("\nPlotting unified feature importance bar chart...")
                    features_to_plot = [item[0] for item in sorted_feature_importances[:N_FEATURES_TO_PLOT]]
                    importances_to_plot = [item[1] for item in sorted_feature_importances[:N_FEATURES_TO_PLOT]]
                    plt.figure(figsize=(10, max(6, N_FEATURES_TO_PLOT * 0.4))); 
                    sns.barplot(x=importances_to_plot, y=features_to_plot, palette="viridis_r")
                    plt.xlabel('Unified Feature Importance (Two-Level Weighted Mean Absolute SHAP)'); 
                    plt.ylabel('Feature'); 
                    plt.title(f'Top {N_FEATURES_TO_PLOT} Feature Importances from Two-Level Weighted SHAP'); 
                    plt.tight_layout()
                    plot_filename = f'unified_feature_importances_2level_weighted_{time.strftime("%Y%m%d_%H%M%S")}.png'
                    plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
                    print(f"Successfully saved unified feature importance chart to: {plot_filename}")
                    plt.show()

                    # 6. Export SHAP data to Excel
                    sorted_shap_feature_names = [name for name, _ in sorted_feature_importances]
                    final_X_data_for_plot_sorted = final_X_data_for_plot[sorted_shap_feature_names]
                    shap_values_df_original_order = pd.DataFrame(final_shap_values_for_plot, columns=feature_names_list)
                    shap_values_df_sorted = shap_values_df_original_order[sorted_shap_feature_names]
                    shap_values_df_sorted.columns = [f'SHAP_{col}' for col in shap_values_df_sorted.columns]
                    combined_shap_data_df = pd.concat([final_X_data_for_plot_sorted.reset_index(drop=True), shap_values_df_sorted.reset_index(drop=True)], axis=1)
                    combined_shap_data_df['SHAP_Base_Value'] = final_base_value
                    excel_filename_shap_data = f'shap_swarm_data_2level_weighted_{time.strftime("%Y%m%d_%H%M%S")}.xlsx'
                    combined_shap_data_df.to_excel(excel_filename_shap_data, index=False, engine='openpyxl')
                    print(f"Successfully exported aggregated SHAP swarm data to: {excel_filename_shap_data} (features sorted by unified importance)")

                    # 7. Plot and save SHAP swarm plot
                    print("\nPlotting SHAP swarm plot...")
                    shap_explanation = shap.Explanation(values=final_shap_values_for_plot, base_values=final_base_value, data=final_X_data_for_plot.values, feature_names=feature_names_list)
                    plt.figure()
                    shap.summary_plot(shap_explanation, final_X_data_for_plot, plot_type="dot", max_display=N_FEATURES_TO_PLOT, show=False)
                    plt.title(f'Two-Level R² Weighted SHAP Values (Aggregated over {valid_seeds_count} Seeds)')
                    plt.tight_layout()
                    shap_plot_filename = f'shap_summary_plot_2level_weighted_{time.strftime("%Y%m%d_%H%M%S")}.png'
                    plt.savefig(shap_plot_filename, dpi=300, bbox_inches='tight')
                    print(f"Successfully saved SHAP swarm plot to: {shap_plot_filename}")
                    plt.show()
                else:
                    print("Warning: Sum of all Level 2 weights (Stacking R²) is zero or invalid. Cannot perform final weighted average for feature importance.")
            else:
                print("Warning: No valid weighted SHAP data was collected. Cannot perform unified analysis.")
        except Exception as e:
            print(f"\nA critical error occurred during the [Two-Level Weighted] SHAP analysis: {e}")
            print(traceback.format_exc())

    # =================================================================================
    # --- End of Analysis Module ---
    # =================================================================================

else:
    print("\nError: No seeds completed successfully. Cannot calculate average results or generate plots.")

print("\n--- Exporting detailed prediction results to Excel (pivoted by model) ---")
if all_plot_data_accumulated:
    try:
        plot_data_df_for_pivot = pd.DataFrame(all_plot_data_accumulated)
        export_df_pivoted = pd.pivot_table(plot_data_df_for_pivot, 
                                           index=['Seed', 'Fold', 'Sample_Index', 'Actual'], 
                                           columns=['Model', 'Set'], 
                                           values='Predicted').reset_index()
        export_df_pivoted.columns = ['_'.join(filter(None, map(str, col))).strip() for col in export_df_pivoted.columns.values]
        excel_filename = f'cv_predictions_pivoted_{time.strftime("%Y%m%d_%H%M%S")}.xlsx'
        export_df_pivoted.to_excel(excel_filename, index=False, engine='openpyxl')
        print(f"Successfully exported pivoted prediction data to: {excel_filename}")
    except Exception as e: print(f"!!! Error exporting pivoted prediction data: {e}")

print("\n--- Exporting detailed R² and RMSE scores per seed/fold to Excel (pivoted by model) ---")
if all_scores_accumulated:
    try:
        scores_df = pd.DataFrame(all_scores_accumulated)
        scores_df_pivoted = scores_df.pivot_table(index=['Seed', 'Fold'], columns=['Model', 'Metric'], values='Value').reset_index()
        scores_df_pivoted.columns = [f'{col[1]}_{col[0]}' if col[1] else col[0] for col in scores_df_pivoted.columns]
        excel_filename_scores = f'cv_model_scores_per_fold_pivoted_{time.strftime("%Y%m%d_%H%M%S")}.xlsx'
        scores_df_pivoted.to_excel(excel_filename_scores, index=False, engine='openpyxl')
        print(f"Successfully exported R² and RMSE scores per seed/fold to: {excel_filename_scores}")
    except Exception as e: print(f"!!! Error exporting R² and RMSE scores per seed/fold: {e}")

print("\n--- Script End ---")

In [ ]:
# Script 5: Prediction
# ==============================================================================
# --- User Configuration ---
# ==============================================================================
# Please modify all tunable parameters in this section.

# --- 1. Prediction File Settings ---
# Excel file name for the unknown data to be predicted.
UNKNOWN_DATA_FILE = 'Gap-prediction.xlsx' # For target1: Gap
# UNKNOWN_DATA_FILE = 'Barrier-prediction.xlsx' # For target2: Barrier
# Define the range to extract features (X_new) from the Excel file (using iloc slicing).
# (slice(None), slice(1, None)) means reading all rows, and from the 2nd column (index 1) to the last column.
# This will skip the first column (index 0) of the Excel file.
UNKNOWN_DATA_FILE_COLUMN_RANGE = (slice(None), slice(1, None))

# --- 2. Model Reuse and Training Settings ---
# Whether to reuse the Stacking model already trained by a previous script.
REUSE_PRETRAINED_STACKING_MODEL = True

# The following parameters are only effective if REUSE_PRETRAINED_STACKING_MODEL = False (i.e., when retraining).
PREDICT_CV_N_SPLITS = 10                  # Number of cross-validation folds for generating OOF predictions.
PREDICT_META_LEARNER_N_SPLITS_CV = 10     # Number of 【splits】 for the Meta-Learner's internal cross-validation.
PREDICT_META_LEARNER_N_REPEATS_CV = 3     # Number of 【repeats】 for the Meta-Learner's internal cross-validation.
PREDICT_META_LEARNER_N_ITER_BAYESIAN = 50 # Number of iterations for Meta-Learner Bayesian Optimization.

# --- 3. Base Learner Selection ---
# !!! IMPORTANT: This list should be consistent with the configuration in the optimization and evaluation scripts !!!
ENABLED_BASE_LEARNERS = [
    'XGBR',   # XGBoost
    'RF',     # Random Forest
    'GBRT',   # Gradient Boosting Regressor
    'HGBR',   # Hist Gradient Boosting
    'ETR',    # Extra Trees Regressor
    'CBR',    # CatBoost
    'LGBM',   # LightGBM
]

# --- 4. Results Export Settings ---
PREDICTION_OUTPUT_FILENAME_PREFIX = 'unknown_predictions'
PREDICTION_EXPORT_TO_EXCEL = True
PREDICTION_EXPORT_TO_CSV = False

# --- 5. General Random State Settings ---
# !!! IMPORTANT: This value should be consistent with DEFAULT_MODEL_RANDOM_STATE in the other two scripts !!!
DEFAULT_MODEL_RANDOM_STATE = 0 # You can change this to any integer you prefer, e.g., 42 or 123


# ==============================================================================
# --- Main Script Body (Usually no modification is needed below) ---
# ==============================================================================

# --- 1. Library Imports and Environment Check ---
import pandas as pd
import numpy as np
import time
import warnings
import os
import re
import sys

import xgboost as XGB
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.base import clone
from skopt import BayesSearchCV
from skopt.space import Real

print("--- Starting Prediction on Unknown Data ---")

try:
    from catboost import CatBoostRegressor
    catboost_available = True
except ImportError:
    catboost_available = False
try:
    from lightgbm import LGBMRegressor
    lightgbm_available = True
except ImportError:
    lightgbm_available = False
print("Library imports and environment check complete.")


# --- 2. Core Variable Loading and Preparation ---
# The mock data fallback is removed; the script will now exit if X, Y, or grid_searches are not defined.
# Assumes X, Y, and grid_searches are provided by upstream scripts (e.g., data prep and model optimization).
if 'X' not in locals() and 'X' not in globals():
    print("Error: Variable 'X' (training feature set) is not defined. Please ensure the data preparation and model optimization scripts have been run first.")
    sys.exit(1)
if 'Y' not in locals() and 'Y' not in globals():
    print("Error: Variable 'Y' (training target set) is not defined. Please ensure the data preparation and model optimization scripts have been run first.")
    sys.exit(1)
if 'grid_searches' not in locals() and 'grid_searches' not in globals():
    print("Error: Variable 'grid_searches' is not defined. Please ensure the model optimization script has been run first.")
    sys.exit(1)

# [Fix] Use a new variable name to store the filtered models to avoid overwriting the original grid_searches.
filtered_grid_searches = {name: gs for name, gs in grid_searches.items() if name in ENABLED_BASE_LEARNERS}
print(f"\nModel filtering complete: Selected {len(filtered_grid_searches)} enabled models from {len(grid_searches)} available models: {list(filtered_grid_searches.keys())}")
if not filtered_grid_searches:
    print("\n!!! Fatal Error: No base learners were selected. Please check the ENABLED_BASE_LEARNERS list in the configuration.")
    sys.exit(1)

# Clean the feature names of the training data X and save the list for alignment with prediction data.
feature_names_list = list(X.columns)
X.columns = [re.sub(r'\[|\]|<', '_', col) for col in feature_names_list]
feature_names_list = list(X.columns) # Update the list to the cleaned names

print(f"\nAttempting to load the unknown prediction set from '{UNKNOWN_DATA_FILE}'...")
if not os.path.exists(UNKNOWN_DATA_FILE):
    print(f"Error: Prediction set file '{UNKNOWN_DATA_FILE}' not found. Please check the file path.")
    sys.exit(1)
else:
    try:
        full_data_from_excel = pd.read_excel(UNKNOWN_DATA_FILE)
        # Slice the data according to the specified column range
        X_new = full_data_from_excel.iloc[UNKNOWN_DATA_FILE_COLUMN_RANGE].copy()
    except Exception as e:
        print(f"Error: Failed to load or process the prediction set '{UNKNOWN_DATA_FILE}': {e}. Please check the file content and column range settings.")
        sys.exit(1)

original_X_new_index = X_new.index # Save original index for results export

# --- Modification: Align column names by position directly, instead of reindexing by name. ---
# Check if the number of features in the sliced prediction set matches the training set.
if X_new.shape[1] != len(feature_names_list):
    print(f"Error: The sliced prediction set X_new has {X_new.shape[1]} columns, but the training set has {len(feature_names_list)} features. The number of features does not match.")
    print("Please check the 'UNKNOWN_DATA_FILE_COLUMN_RANGE' setting to ensure it reads the correct number of features.")
    sys.exit(1)

# Directly assign the cleaned feature names from the training set to X_new's columns.
# This assumes that the order of features in X_new is identical to the order in the training data X.
# This avoids NaN issues caused by minor differences in column names.
X_new.columns = feature_names_list

# Check for NaNs that might have existed in the original X_new data.
if X_new.isnull().any().any():
    missing_cols_in_raw_X_new = X_new.columns[X_new.isnull().any()].tolist()
    print(f"Warning: The raw prediction set X_new contains missing values (NaN). Columns with missing values: {missing_cols_in_raw_X_new}")
    # X_new = X_new.fillna(0) # Optional filling strategy; uncomment if you need to fill NaNs.
# --------------------------------------------------------------------------------

print(f"\nTraining data X shape: {X.shape}")
print(f"Prediction data X_new shape (after assigning training column names): {X_new.shape}")


# --- 3. Core Functions and Model Training ---
def initialize_best_estimators(grid_searches_dict):
    """Initializes base learner instances with the best found parameters."""
    estimators_init = {}
    available_models = {
        'XGBR': (XGB.XGBRegressor, {'objective': 'reg:squarederror', 'random_state': DEFAULT_MODEL_RANDOM_STATE}),
        'RF': (RandomForestRegressor, {'random_state': DEFAULT_MODEL_RANDOM_STATE}),
        'GBRT': (GradientBoostingRegressor, {'random_state': DEFAULT_MODEL_RANDOM_STATE}),
        'ETR': (ExtraTreesRegressor, {'random_state': DEFAULT_MODEL_RANDOM_STATE}),
        'HGBR': (HistGradientBoostingRegressor, {'random_state': DEFAULT_MODEL_RANDOM_STATE})
    }
    if catboost_available: available_models['CBR'] = (CatBoostRegressor, {'verbose': False, 'random_state': DEFAULT_MODEL_RANDOM_STATE, 'allow_writing_files': False})
    if lightgbm_available: available_models['LGBM'] = (LGBMRegressor, {'random_state': DEFAULT_MODEL_RANDOM_STATE, 'verbosity': -1, 'objective': 'regression'})

    print("Initializing base learners...")
    for name in grid_searches_dict.keys():
        if name not in available_models:
            print(f"  Warning: Model {name} is not in the list of available models. Skipping initialization.")
            continue
        model_class, fixed_params = available_models[name]
        if name in grid_searches_dict and grid_searches_dict[name] is not None and hasattr(grid_searches_dict[name], 'best_estimator_'):
            try:
                estimators_init[name] = grid_searches_dict[name].best_estimator_
                print(f"  Successfully initialized {name} with optimized parameters.")
            except Exception as e:
                print(f"  Error initializing {name} from best_estimator_: {e}. This model will be skipped.")
        else:
            print(f"  Warning: Optimized results for {name} not found. Initializing with default parameters.")
            try:
                estimators_init[name] = model_class(**fixed_params)
                print(f"  Successfully initialized {name} with default parameters.")
            except Exception as e:
                print(f"  Error initializing {name} with default parameters: {e}. This model will be skipped.")
    initialized_estimators = {k: v for k, v in estimators_init.items() if v}
    if not initialized_estimators: print("Warning: No models were successfully initialized!")
    return initialized_estimators

meta_learner_params = {
    'elasticnet__alpha': Real(1e-5, 10.0, prior='log-uniform', name='elasticnet__alpha'),
    'elasticnet__l1_ratio': Real(0.0, 1.0, prior='uniform', name='elasticnet__l1_ratio')
}
final_meta_learner = None; base_estimators_for_final_training = None

if REUSE_PRETRAINED_STACKING_MODEL:
    print("\n--- Attempting to reuse the previously trained Stacking model ---")
    if 'global_best_meta_learner' in locals() and global_best_meta_learner is not None and hasattr(global_best_meta_learner, 'predict'):
        final_meta_learner = global_best_meta_learner
        print("Successfully reused the previously trained meta-learner.")
    else:
        print("Warning: No valid 'global_best_meta_learner' found. Falling back to retraining mode.")
        REUSE_PRETRAINED_STACKING_MODEL = False

if not REUSE_PRETRAINED_STACKING_MODEL:
    print("\n--- Retraining the final Stacked model ---")
    base_estimators_for_oof = initialize_best_estimators(filtered_grid_searches)
    if not base_estimators_for_oof:
        print("Error: No base learners available to train the Stacked model."); sys.exit(1)

    model_names_available = list(base_estimators_for_oof.keys())
    oof_preds_for_meta_training = np.zeros((len(X), len(model_names_available)))
    
    kf_for_oof = KFold(n_splits=PREDICT_CV_N_SPLITS, shuffle=True, random_state=DEFAULT_MODEL_RANDOM_STATE)

    print(f"Starting {PREDICT_CV_N_SPLITS}-fold cross-validation to generate OOF predictions (Random Seed: {DEFAULT_MODEL_RANDOM_STATE})...")
    for fold_idx, (train_idx, val_idx) in enumerate(kf_for_oof.split(X, Y)):
        print(f"  Fold {fold_idx+1}/{PREDICT_CV_N_SPLITS}")
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        Y_train_fold = Y.iloc[train_idx]
        for i, (name, estimator_template) in enumerate(base_estimators_for_oof.items()):
            estimator_fold = clone(estimator_template)
            try:
                estimator_fold.fit(X_train_fold, Y_train_fold)
                oof_preds_for_meta_training[val_idx, i] = estimator_fold.predict(X_val_fold)
            except Exception as e:
                print(f"    Warning: Model {name} failed during fit/predict: {e}. OOF predictions will be zero.")

    if np.all(oof_preds_for_meta_training == 0) or np.any(np.isnan(oof_preds_for_meta_training)):
        print("Error: OOF predictions are invalid. Cannot train meta-learner. Please check if base learners are training correctly."); sys.exit(1)

    print("Starting Bayesian Optimization and training for the meta-learner...")
    meta_bayes_search = BayesSearchCV(
        estimator=Pipeline([('scaler', StandardScaler()), ('elasticnet', ElasticNet(random_state=DEFAULT_MODEL_RANDOM_STATE, max_iter=2000))]),
        search_spaces=meta_learner_params, n_iter=PREDICT_META_LEARNER_N_ITER_BAYESIAN, scoring='r2',
        cv=RepeatedKFold(n_splits=PREDICT_META_LEARNER_N_SPLITS_CV, n_repeats=PREDICT_META_LEARNER_N_REPEATS_CV, random_state=DEFAULT_MODEL_RANDOM_STATE),
        n_jobs=-1, random_state=DEFAULT_MODEL_RANDOM_STATE, verbose=1)
    try:
        meta_bayes_search.fit(oof_preds_for_meta_training, Y)
        final_meta_learner = meta_bayes_search.best_estimator_
        print("\n--- Meta-learner training complete ---")
        print(f"Meta-Learner Best Score (CV R² on Full OOF): {meta_bayes_search.best_score_:.4f}")
        print(f"Meta-Learner Best Parameters: {dict(meta_bayes_search.best_params_)}")

        if final_meta_learner and 'elasticnet' in final_meta_learner.named_steps:
            elastic_net_model = final_meta_learner.named_steps['elasticnet']
            if hasattr(elastic_net_model, 'coef_'):
                print("\n--- Base Learner Coefficients in Meta-Learner (ElasticNet) ---")
                for i, name in enumerate(model_names_available):
                    if i < len(elastic_net_model.coef_):
                        print(f"  {name}: {elastic_net_model.coef_[i]:.6f}")
                    else:
                        print(f"  Warning: Coefficient for model {name} not found (index out of bounds).")
            else:
                print("Warning: Meta-learner (ElasticNet) has no 'coef_' attribute. Cannot display coefficients.")
        else:
            print("Warning: Meta-learner or its ElasticNet component not found. Cannot display coefficients.")

    except Exception as e:
        print(f"\n!!! Meta-learner training failed: {e}. Cannot proceed with prediction."); sys.exit(1)

print("\n--- Training final base learners on the entire training dataset ---")
base_estimators_for_final_training = initialize_best_estimators(filtered_grid_searches)
for name, estimator in base_estimators_for_final_training.items():
    print(f"  Training the final {name} model...")
    try:
        estimator.fit(X, Y)
    except Exception as e:
        print(f"    Warning: Failed to train the final {name} model: {e}. This model will be removed from the prediction process.")
        base_estimators_for_final_training[name] = None
base_estimators_for_final_training = {k: v for k, v in base_estimators_for_final_training.items() if v}


# --- 4. Predicting on Unknown Data ---
if not final_meta_learner or not base_estimators_for_final_training:
    print("Fatal Error: The Stacked model could not be built successfully. Cannot make predictions."); sys.exit(1)

print("\n--- Making Stacked predictions on the unknown data ---")
base_predictions_on_new_data, base_model_names_for_prediction = [], []
print("Generating predictions from base learners on the new data...")
for name, estimator in base_estimators_for_final_training.items():
    try:
        pred = estimator.predict(X_new)
        base_predictions_on_new_data.append(pred)
        base_model_names_for_prediction.append(name)
        print(f"  {name} prediction complete.")
    except Exception as e:
        print(f"  Error while predicting with model {name} on new data: {e}. This model will be skipped.")

if not base_predictions_on_new_data:
    print("Error: All base learners failed to predict on the new data."); sys.exit(1)

meta_features_for_prediction = np.column_stack(base_predictions_on_new_data)
print(f"Shape of base learner predictions (input for meta-learner): {meta_features_for_prediction.shape}")

print("Making final Stacked prediction using the meta-learner...")
try:
    final_stacked_predictions = final_meta_learner.predict(meta_features_for_prediction)
    print("Final Stacked prediction complete.")
except Exception as e:
    print(f"Error during final prediction by meta-learner: {e}"); sys.exit(1)


# --- 5. Organizing and Exporting Prediction Results ---
print("\n--- Organizing and exporting prediction results ---")
results_df = pd.DataFrame(index=original_X_new_index) # Use the original index
for i, name in enumerate(base_model_names_for_prediction):
    results_df[f'{name}_Prediction'] = base_predictions_on_new_data[i]
results_df['Stacked_Prediction'] = final_stacked_predictions

current_timestamp = time.strftime("%Y%m%d_%H%M%S")
output_filename_base = f"{PREDICTION_OUTPUT_FILENAME_PREFIX}_{current_timestamp}"

if PREDICTION_EXPORT_TO_EXCEL:
    excel_filename = f"{output_filename_base}.xlsx"
    try:
        results_df.to_excel(excel_filename, index=True, engine='openpyxl')
        print(f"Prediction results successfully exported to Excel: {excel_filename}")
    except ImportError:
        print("!!! Exporting to Excel requires the 'openpyxl' library. Switching to CSV export.")
        PREDICTION_EXPORT_TO_EXCEL = False
    except Exception as e:
        print(f"!!! Error exporting prediction results to Excel: {e}")
        PREDICTION_EXPORT_TO_EXCEL = False

if PREDICTION_EXPORT_TO_CSV or not PREDICTION_EXPORT_TO_EXCEL:
    csv_filename = f"{output_filename_base}.csv"
    try:
        results_df.to_csv(csv_filename, index=True)
        print(f"Prediction results successfully exported to CSV: {csv_filename}")
    except Exception as e:
        print(f"!!! Error exporting prediction results to CSV: {e}")

print("\nPrediction script for unknown data has finished.")